In [146]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd

In [147]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [148]:
folder = "memr_4_at_25#10#2019_res-0.2"
file = "defbuffer1mem4.csv"
df=pd.read_csv("{}/{}".format(folder, file), dtype='object', comment='#')
df['Fractional Seconds']=df['Fractional Seconds'].astype('object')
step_time = 90
first_level_time = 15
#print(df.info())

In [149]:
e=df['Fractional Seconds'][0]
df['Fractional Seconds']=df['Fractional Seconds'].map(lambda x:int(float(x)*10**(len(x)-1))//1000)

In [150]:
df['Reading']=df['Reading'].astype('float64')
df['Value']=df['Value'].astype('float64')
#print(df.info())

In [151]:
from datetime import datetime as dt
import numpy as np
arr=[dt.strptime(df['Date'][i]+" "+df['Time'][i]+" "+str(df['Fractional Seconds'][i]),"%m/%d/%Y %H:%M:%S %f") for i in range(len(df['Date']))]
data=np.array(arr)
df['Datetime']=pd.Series(data)

In [152]:
df['TimeDelta']=(df['Datetime']-df['Datetime'][0])
df['TimeDelta']=df['TimeDelta'].map(lambda x:x.total_seconds())

In [153]:
df['Resistance']=df['Value']/df['Reading']
#print(df['Resistance'])

In [154]:
temp_df=pd.DataFrame(columns=["Value","Reading","TimeDelta"])
one_point=pd.DataFrame(columns=["Value","Reading","TimeDelta"])
l = [a * 0.1 for a in range(0, 11)]
l += [a * 0.1 for a in range(9, -11,-1)]
l += [a * 0.1 for a in range(-9, 1)]

for idx,elem in enumerate(l):
    temp_df=df[(df['Value'] >=elem-0.05) & (df['Value'] < elem+0.05)& (df['TimeDelta'] < (idx*step_time +first_level_time))]    
    one_point=one_point.append(temp_df.iloc[-1,:])   
print(one_point)
# temp_df.head(10)

          Value       Reading    TimeDelta        Date  \
309   -0.000013 -8.206862e-08    14.870849  10/25/2019   
2181   0.100018  1.166683e-07   104.999783  10/25/2019   
4057   0.200008  2.943158e-07   194.884323  10/25/2019   
5931   0.300017  4.861735e-07   284.881844  10/25/2019   
7803   0.400008  7.145196e-07   374.888483  10/25/2019   
9673   0.500044  9.779831e-07   464.848992  10/25/2019   
11550  0.600030  1.268213e-06   554.884585  10/25/2019   
13423  0.700028  1.592509e-06   644.856256  10/25/2019   
15295  0.800053  1.911672e-06   734.892833  10/25/2019   
17165  0.900040  2.199610e-06   824.850909  10/25/2019   
19041  1.000068  2.479024e-06   914.882459  10/25/2019   
20915  0.900049  2.010153e-06  1004.864560  10/25/2019   
22790  0.800060  1.640926e-06  1094.864618  10/25/2019   
24661  0.700042  1.342384e-06  1184.886151  10/25/2019   
26533  0.600049  1.082500e-06  1274.867661  10/25/2019   
28408  0.500063  8.435351e-07  1364.867017  10/25/2019   
30284  0.40003

In [155]:
t1=step_time*10+first_level_time
t2=step_time*20+first_level_time
t3=step_time*30+first_level_time
s1=step_time*10
s2=step_time*20
s3=step_time*30
datas =[go.Scatter(
        x=one_point["Value"][one_point["TimeDelta"]<t1],
        y=one_point["Reading"][one_point["TimeDelta"]<t1],
        name='hyst1'
        ),
        go.Scatter(
        x=one_point["Value"][(one_point["TimeDelta"]>s1)&(one_point["TimeDelta"]<t2)],
        y=one_point["Reading"][(one_point["TimeDelta"]>s1)&(one_point["TimeDelta"]<t2)],
        name='hyst2'
        ),
        go.Scatter(
        x=one_point["Value"][(one_point["TimeDelta"]>s2)&(one_point["TimeDelta"]<t3)],
        y=one_point["Reading"][(one_point["TimeDelta"]>s2)&(one_point["TimeDelta"]<t3)],
        name='hyst3'
        ),
        go.Scatter(
        x=one_point["Value"][one_point["TimeDelta"]>s3],
        y=one_point["Reading"][one_point["TimeDelta"]>s3],
        name='hyst4'
        )]
layout = go.Layout(
    title='{}_hyst'.format(file),
    xaxis=dict(
        title="volts",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title="amps",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    autosize=False,
    width=1920,
    height=1080
)

# cоздаем объект Figure и визуализируем его
fig = go.Figure(data=datas, layout=layout)
# fig.write_image("{}/{}_hyst.png".format(folder,file))
#iplot(fig, show_link=False)
plotly.offline.plot(fig,filename="{}/{}_hyst.html".format(folder,file),show_link=False, auto_open=False)

'memr_4_at_25#10#2019_res-0.2/defbuffer1mem4.csv_hyst.html'

In [156]:
datas =[go.Scatter(
        x=df['TimeDelta'],
        y=df['Reading'])]
layout = go.Layout(
    title="{}_curr".format(folder),
    xaxis=dict(
        title="TIME",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title="AMPS",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    )
)

# cоздаем объект Figure и визуализируем его
fig1 = go.Figure(data=datas, layout=layout)
#iplot(fig, show_link=False)
plotly.offline.plot(fig1,filename="{}/{}_curr.html".format(folder, file),show_link=False)

'memr_4_at_25#10#2019_res-0.2/defbuffer1mem4.csv_curr.html'

In [157]:
datas =[go.Scatter(
        x=df['TimeDelta'],
        y=df['Value'])]
layout = go.Layout(
    title="{}_volts".format(folder),
    xaxis=dict(
        title="TIME",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title="VOLTS",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    )
)

# cоздаем объект Figure и визуализируем его
fig1 = go.Figure(data=datas, layout=layout)
#iplot(fig, show_link=False)
plotly.offline.plot(fig1,filename="{}/{}_volts.html".format(folder, file),show_link=False)

'memr_4_at_25#10#2019_res-0.2/defbuffer1mem4.csv_volts.html'

In [158]:
volts = []
for i in one_point["Value"]:
    volts.append(round(i, 1))

amp = []
for j in one_point["Reading"]:
    amp.append(j)

res = []
for t in range(0, 41):
    x = round(volts[t] / amp[t])
    if x < 0:
        x *= -1
    if x != 0:
        res.append(x)
        
time = []
for y in one_point["TimeDelta"]:
    time.append(y)
time.pop(0)
time.pop(19)
time.pop()

3614.849955

In [159]:
datas =[go.Scatter(
        x=time,
        y=res,
        name='resistance_{}'.format(file)
        )]
layout = go.Layout(
    title='{}_resistance'.format(file),
    xaxis=dict(
        title="time",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title="resistance",
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    autosize=False,
    width=1920,
    height=1080
)

# cоздаем объект Figure и визуализируем его
fig = go.Figure(data=datas, layout=layout)
# fig.write_image("{}/{}_volt.png".format(folder,file))
#iplot(fig, show_link=False)
plotly.offline.plot(fig,filename="{}/{}_resistance.html".format(folder,file),show_link=False, auto_open=False)

'memr_4_at_25#10#2019_res-0.2/defbuffer1mem4.csv_resistance.html'